<a href="https://colab.research.google.com/github/alephpt/AI_Projects/blob/main/GPT_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as fn
import requests

In [ ]:
batch_size = 16 # Number of Sequences to Process in Parallel
block_size = 32 # Max Content Length for Predictions
max_iterations = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iterations = 200 # Number of iterations before saving state
n_embed = 64 # Maps the input sequence into high-dimensional space
n_head = 4 # Attention head for multi-head attention layers
n_layers = 4
dropout = 0.0

In [ ]:
torch.manual_seed(1337)
bible_url = 'https://www.o-bible.com/download/kjv.txt'
bible = requests.get(bible_url)
bible_text = bible.text.split("\n")
bible_text.remove(bible_text[0])

In [ ]:
print(bible_text[0])

Ge1:1 In the beginning God created the heaven and the earth.


In [ ]:
vocab = sorted(list(set(bible.text)))
vocab_size = len(vocab)
print("Vocab Size:", vocab_size)

Vocab Size: 73


In [ ]:
stoi = { ch: i for i, ch in enumerate(vocab)}
itos = { i: ch for i, ch in enumerate(vocab)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

nums = encode("Today is the day.")
words = decode(nums)

print("nums", nums)
print("words", words)

nums [41, 61, 50, 47, 71, 1, 55, 65, 1, 66, 54, 51, 1, 50, 47, 71, 8]
words Today is the day.


In [ ]:
data = torch.tensor(encode(bible.text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([4404443]) torch.int64
tensor([29, 61, 58, 71,  1, 23, 55, 48, 58, 51,  6,  1, 22, 67, 66, 54, 61, 64,
        55, 72, 51, 50,  1,  4, 32, 55, 60, 53,  1, 31, 47, 59, 51, 65,  5,  1,
        43, 51, 64, 65, 55, 61, 60,  6,  1, 41, 51, 70, 66, 52, 55, 58, 51,  1,
        18, 12,  9, 10,  9, 14,  8,  0, 28, 51, 10, 19, 10,  1, 30, 60,  1, 66,
        54, 51,  1, 48, 51, 53, 55, 60, 60, 55, 60, 53,  1, 28, 61, 50,  1, 49,
        64, 51, 47, 66, 51, 50,  1, 66, 54, 51])


In [ ]:
n = int(0.9 * len(data)) # train on the first 90%
training_data = data[:n]
validation_data = data[n:]
block_size = 8
training_data[:block_size + 1]
x = training_data[:block_size]
y = training_data[1:block_size + 1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([29]) the target: 61
when input is tensor([29, 61]) the target: 58
when input is tensor([29, 61, 58]) the target: 71
when input is tensor([29, 61, 58, 71]) the target: 1
when input is tensor([29, 61, 58, 71,  1]) the target: 23
when input is tensor([29, 61, 58, 71,  1, 23]) the target: 55
when input is tensor([29, 61, 58, 71,  1, 23, 55]) the target: 48
when input is tensor([29, 61, 58, 71,  1, 23, 55, 48]) the target: 58


In [ ]:
def get_batch(split):
  data = training_data if split == "train" else validation_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i + block_size] for i in ix])
  y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
  return x, y

In [ ]:
xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

inputs:
torch.Size([16, 8])
tensor([[65, 47, 64,  6,  1, 61, 64,  1],
        [ 1, 54, 51, 47, 64, 20,  1, 65],
        [71, 19,  1, 52, 61, 64,  1, 66],
        [48, 61, 69, 51, 50,  1, 54, 55],
        [51, 51,  6,  1, 47, 60, 50,  1],
        [69, 51, 64, 51,  1, 60, 47, 59],
        [67, 60, 50, 47, 66, 55, 61, 60],
        [58, 51, 66,  1, 66, 54, 55, 60],
        [ 1, 55, 52,  1, 66, 54, 61, 67],
        [60, 50,  1, 66, 54, 51, 71,  1],
        [54, 51, 64, 48, 19,  1, 47, 60],
        [51,  1, 65, 61, 60, 65,  1, 61],
        [64,  1, 28, 61, 50,  8,  0, 29],
        [19, 18,  1, 22, 60, 50,  1, 28],
        [33, 36, 39, 25,  1, 61, 67, 64],
        [71, 62, 66,  6,  1, 52, 64, 61]])
targets:
torch.Size([16, 8])
tensor([[47, 64,  6,  1, 61, 64,  1, 60],
        [54, 51, 47, 64, 20,  1, 65, 47],
        [19,  1, 52, 61, 64,  1, 66, 54],
        [61, 69, 51, 50,  1, 54, 55, 59],
        [51,  6,  1, 47, 60, 50,  1, 49],
        [51, 64, 51,  1, 60, 47, 59, 51],
        [60, 50, 4

In [ ]:
for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t + 1]
    target = yb[b, t]
    print(f"when input is {context.tolist()} the target: {target}")

In [35]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iterations)
        for k in range(eval_iterations):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embed, head_size, bias=False)
    self.query = nn.Linear(n_embed, head_size, bias=False)
    self.value = nn.Linear(n_embed, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x)
    q = self.query(x)
    
    wei = q @ k.transpose(-2, -1) * C**-0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
    wei = fn.softmax(wei, -1)
    wei = self.dropout(wei)

    v = self.value(x)

    return wei @ v

In [ ]:
class MultiHeadAtt(nn.Module):
  def __init__(self, n_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
    self.proj = nn.Linear(n_embed, n_embed)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], -1)
    out = self.dropout(self.proj(out))
    return out

In [21]:
class FeedForward(nn.Module):
  def __init__(self, n_embed):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embed, n_embed),
        nn.ReLU(),
    )
  
  def forward(self, x):
    return self.net(x)

In [23]:
class Block(nn.Module):
  def __init__(self, n_embed, n_head):
    super().__init__()
    head_size = n_embed // n_head
    self.sa = MultiHeadAtt(n_head, head_size)
    self.ffwd = FeedForward(n_embed)

  def forward(self):
    x = x + self.sa(x)
    x = x + self.ffwd(x)
    return x

In [39]:
class Bigram(nn.Module):
  def __init__(self, n_embed, n_head):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    self.blocks = nn.Sequential(*[Block(n_embed, n_head) for _ in range(n_layers)])
    self.ln_f = nn.LayerNorm(n_embed)
    self.lm_head = nn.Linear(n_embed, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape

    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))

    x = tok_emb + pos_emb
    x = self.blocks(x)
    x = self.ln_f(x)

    logits = self.lm_head(x)

    if targets == None:
      loss = targets
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, c)
      targets = targets.view(B*T)
      loss = fn.cross_entropy(logits, targets)

    return logits, loss
  
  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_cond = idx[:, -block_size:]
      logits, loss = self(idx_cond)
      logits = logits[:, -1, :]
      probs = fn.softmax(logits, -1)
      idx_next = torch.multinomial(probs, 1)
      idx = torch.cat((idx, idx_next), 1)
    return idx

In [42]:
model = Bigram(n_embed, n_head)
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e3, 'K parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for i in range(max_iterations):
  if i % eval_iterations == 0 or i == max_iterations - 1:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batch('train')

  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backwards()
  optimizer.step()

context = torch.zero((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

  

92.489 K parameters


TypeError: ignored